In [8]:
import pandas as pd
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from operator import itemgetter
import xgboost as xgb
import random
import time
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
from sklearn import preprocessing

from sklearn.metrics import roc_curve, auc,recall_score,precision_score

#paar functions verstehe ich den sinn nicht zB create_feature_map
#habe paar befehle ausgeklammert weil die errors throwen line 90 und 93
# line 127-130 musste  ich die data types ändern weil numerai seine data verändert hat seit 
# 2017 anscheinend

#das sind alle änderungen die ich gemacht habe am ende kommt ein prediction directory raus aber
# glaube die werte sind schwachsinn 




    
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()


def get_importance(gbm, features):
    create_feature_map(features)
    importance = gbm.get_fscore(fmap='xgb.fmap')
    importance = sorted(importance.items(), key=itemgetter(1), reverse=True)
    return importance

def get_features(train, test):
    trainval = list(train.columns.values)
    output = trainval
    return sorted(output)


def run_single(train, test, features, target, random_state=0):
    eta = 0.1
    max_depth= 5 
    subsample = 1
    colsample_bytree = 1
    min_chil_weight=1
    start_time = time.time()

    print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth, subsample, colsample_bytree))
    params = {
        "objective": "reg:logistic",
        "booster" : "gbtree",
        "eval_metric": "logloss",
        "eta": eta,
        "tree_method": 'exact',
        "max_depth": max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent": 1,
        "min_chil_weight":min_chil_weight,
        "seed": random_state,
        #"num_class" : 22,
    }
    num_boost_round = 500
    early_stopping_rounds = 20
    test_size = 0.1

   
    
    X_train, X_valid = train_test_split(train, test_size=test_size, random_state=random_state)
    print('Length train:', len(X_train.index))
    print('Length valid:', len(X_valid.index))
    y_train = X_train[target]
    y_valid = X_valid[target]
    dtrain = xgb.DMatrix(X_train[features], y_train, missing=-99, enable_categorical = True)
    dvalid = xgb.DMatrix(X_valid[features], y_valid, missing =-99, enable_categorical = True)

    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

    print("Validating...")
    check = gbm.predict(xgb.DMatrix(X_valid[features]), ntree_limit=gbm.best_iteration+1)
    
    #area under the precision-recall curve
    score = mean_squared_error(X_valid[target].values, check)
    print('mean_squared_error: {:.6f}'.format(score))

    
    check2=check.round()
    #score = precision_score(X_valid[target].values, check2)
    print('precision score: {:.6f}'.format(score))

    #score = recall_score(X_valid[target].values, check2)
    print('recall score: {:.6f}'.format(score))
    
    imp = get_importance(gbm, features)
    print('Importance array: ', imp)

    print("Predict test set... ")
    test_prediction = gbm.predict(xgb.DMatrix(test[features],missing = -99), ntree_limit=gbm.best_iteration+1)
    score = mean_squared_error(test[target].values, test_prediction)

    print('mean squared error test set: {:.6f}'.format(score))
    


    print('Training time: {} minutes'.format(round((time.time() - start_time)/60, 2)))
    return test_prediction, imp, gbm.best_iteration+1
    
    
    
    
    
    
    
    
    
    
    
if __name__ == "__main__":
    start_time = dt.datetime.now()
    print("Start time: ",start_time)
  

    train = pd.read_csv('numerai_datasets/numerai_training_data.csv')
    lbl = preprocessing.LabelEncoder()
    train['id'] = lbl.fit_transform(train['id'].astype(str))
    train['era'] = lbl.fit_transform(train['era'].astype(str))
    train['data_type'] = lbl.fit_transform(train['data_type'].astype(str))
    print(train.isnull().values.any())
    features=list(train.columns)

    features.remove('target')
    
    #rem_list=['feature19','feature12','feature21','feature24']
    
    #for each in rem_list:
    #    features.remove(each)
    #print(train['target'])
 
    train, test = train_test_split(train, test_size=.1, random_state=random.seed(2016))


    print(features)

    print("Building model.. ",dt.datetime.now()-start_time)
    preds, imp, num_boost_rounds = run_single(train, test, features, 'target',42)
 
    print(dt.datetime.now()-start_time)
 

Start time:  2021-05-05 21:26:20.227669
False
['id', 'era', 'data_type', 'feature_intelligence1', 'feature_intelligence2', 'feature_intelligence3', 'feature_intelligence4', 'feature_intelligence5', 'feature_intelligence6', 'feature_intelligence7', 'feature_intelligence8', 'feature_intelligence9', 'feature_intelligence10', 'feature_intelligence11', 'feature_intelligence12', 'feature_charisma1', 'feature_charisma2', 'feature_charisma3', 'feature_charisma4', 'feature_charisma5', 'feature_charisma6', 'feature_charisma7', 'feature_charisma8', 'feature_charisma9', 'feature_charisma10', 'feature_charisma11', 'feature_charisma12', 'feature_charisma13', 'feature_charisma14', 'feature_charisma15', 'feature_charisma16', 'feature_charisma17', 'feature_charisma18', 'feature_charisma19', 'feature_charisma20', 'feature_charisma21', 'feature_charisma22', 'feature_charisma23', 'feature_charisma24', 'feature_charisma25', 'feature_charisma26', 'feature_charisma27', 'feature_charisma28', 'feature_charisma

[9]	train-logloss:0.69256	eval-logloss:0.69309
[10]	train-logloss:0.69254	eval-logloss:0.69309
[11]	train-logloss:0.69252	eval-logloss:0.69309
[12]	train-logloss:0.69250	eval-logloss:0.69308
[13]	train-logloss:0.69248	eval-logloss:0.69307
[14]	train-logloss:0.69247	eval-logloss:0.69307
[15]	train-logloss:0.69245	eval-logloss:0.69307
[16]	train-logloss:0.69243	eval-logloss:0.69306
[17]	train-logloss:0.69241	eval-logloss:0.69305
[18]	train-logloss:0.69240	eval-logloss:0.69305
[19]	train-logloss:0.69238	eval-logloss:0.69305
[20]	train-logloss:0.69237	eval-logloss:0.69304
[21]	train-logloss:0.69235	eval-logloss:0.69303
[22]	train-logloss:0.69234	eval-logloss:0.69303
[23]	train-logloss:0.69232	eval-logloss:0.69303
[24]	train-logloss:0.69230	eval-logloss:0.69303
[25]	train-logloss:0.69229	eval-logloss:0.69303
[26]	train-logloss:0.69228	eval-logloss:0.69303
[27]	train-logloss:0.69226	eval-logloss:0.69303
[28]	train-logloss:0.69224	eval-logloss:0.69302
[29]	train-logloss:0.69223	eval-logloss:0

/opt/anaconda3/lib/python3.8/site-packages/xgboost/core.py:101: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


mean_squared_error: 0.049903
precision score: 0.049903
recall score: 0.049903
Importance array:  [('id', 102), ('era', 98), ('feature_dexterity6', 34), ('feature_charisma63', 31), ('feature_dexterity7', 28), ('feature_dexterity11', 25), ('feature_dexterity4', 24), ('feature_strength34', 23), ('feature_wisdom35', 23), ('feature_intelligence1', 23), ('feature_dexterity14', 23), ('feature_charisma85', 22), ('feature_wisdom42', 19), ('feature_wisdom23', 18), ('feature_intelligence4', 18), ('feature_wisdom13', 18), ('feature_strength19', 17), ('feature_strength4', 17), ('feature_intelligence3', 17), ('feature_intelligence11', 16), ('feature_charisma10', 16), ('feature_strength22', 16), ('feature_wisdom22', 16), ('feature_charisma19', 15), ('feature_dexterity3', 15), ('feature_intelligence9', 15), ('feature_charisma13', 15), ('feature_constitution52', 14), ('feature_strength3', 14), ('feature_intelligence8', 14), ('feature_dexterity9', 14), ('feature_strength35', 14), ('feature_intelligence5

/opt/anaconda3/lib/python3.8/site-packages/xgboost/core.py:101: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


mean squared error test set: 0.049533
Training time: 6.86 minutes
0:07:21.325598


In [6]:
import pandas as pd

df = pd.read_csv('numerai_datasets/numerai_training_data.csv')
print(df.dtypes)

id                        object
era                       object
data_type                 object
feature_intelligence1    float64
feature_intelligence2    float64
                          ...   
feature_wisdom43         float64
feature_wisdom44         float64
feature_wisdom45         float64
feature_wisdom46         float64
target                   float64
Length: 314, dtype: object
